### CHibuzo Ukegbu

# CS534 Homework 3

Put your homework in the directory with your name. Please mentionin this file the names of any students with whom you collaborated. If you didn't collaborate with anyone, mark your collaborators as "None." Remember, your goal is to communicate. Full credit will be given only to correct solutions which are described clearly. Convoluted and obtuse descriptions will receive low marks. To complete your homework, you may ONLY consult the following material:

lecture slides course notes you or others took during lecture. the required text (CLRS) websites that may clarify the concepts covered in the material but do not in any way provide complete solutions to the problems. Deadline 04/27/2020

Please provide an answer to the following question:

## Basic Setup


In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix,accuracy_score
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics.cluster import homogeneity_score
import warnings
warnings.filterwarnings('ignore')

# Question 1 (10 pts)
Write the explicit constraints (without using any vectorial notation, as a summation of single variables multiplied by a constant + bias term) of the Support Vector Machine to classify correctly iris dataset (Iris-Versicolor vs. others). In particular use 5 points in Iris-Versicolor, 2 points for iris-setosa, and 3 points for iris Virginia. Please show the points you selected and after the constraints

In [2]:
X, y = datasets.load_iris(return_X_y=True)
classes =  list(datasets.load_iris()['target_names'])

print("Iris Versicolor points:")
data_vers = X[y==classes.index("versicolor")][:5]
print(data_vers)
print()
print("Iris Setosa points:")
data_seto = X[y==classes.index("setosa")][:2]
print(data_seto)
print()
print("Iris Virginica points:")
data_virg = X[y==classes.index("virginica")][:3]
print(data_virg)

# Applying the constraints
print("\nThe explicit contraints of the SVM to classify iris dataset correctly are as follows:")
# Versicolor
for i in range(data_vers.shape[0]):
    for a, feat in enumerate(data_vers[i, :]):
        print("{}*{} + ".format(feat, "a"+str(a)), end="")
        print("b >= 1")
# Setosa
for i in range(data_seto.shape[0]):
    for a, feat in enumerate(data_seto[i, :]):
        print("{}*{} + ".format(feat, "a"+str(a)), end="")
        print("b <= -1")
# Virginica
for i in range(data_virg.shape[0]):
    for a, feat in enumerate(data_virg[i, :]):
        print("{}*{} + ".format(feat, "a"+str(a)), end="")
        print("b <= -1")

Iris Versicolor points:
[[7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.9 1.5]
 [5.5 2.3 4.  1.3]
 [6.5 2.8 4.6 1.5]]

Iris Setosa points:
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]]

Iris Virginica points:
[[6.3 3.3 6.  2.5]
 [5.8 2.7 5.1 1.9]
 [7.1 3.  5.9 2.1]]

The explicit contraints of the SVM to classify iris dataset correctly are as follows:
7.0*a0 + b >= 1
3.2*a1 + b >= 1
4.7*a2 + b >= 1
1.4*a3 + b >= 1
6.4*a0 + b >= 1
3.2*a1 + b >= 1
4.5*a2 + b >= 1
1.5*a3 + b >= 1
6.9*a0 + b >= 1
3.1*a1 + b >= 1
4.9*a2 + b >= 1
1.5*a3 + b >= 1
5.5*a0 + b >= 1
2.3*a1 + b >= 1
4.0*a2 + b >= 1
1.3*a3 + b >= 1
6.5*a0 + b >= 1
2.8*a1 + b >= 1
4.6*a2 + b >= 1
1.5*a3 + b >= 1
5.1*a0 + b <= -1
3.5*a1 + b <= -1
1.4*a2 + b <= -1
0.2*a3 + b <= -1
4.9*a0 + b <= -1
3.0*a1 + b <= -1
1.4*a2 + b <= -1
0.2*a3 + b <= -1
6.3*a0 + b <= -1
3.3*a1 + b <= -1
6.0*a2 + b <= -1
2.5*a3 + b <= -1
5.8*a0 + b <= -1
2.7*a1 + b <= -1
5.1*a2 + b <= -1
1.9*a3 + b <= -1
7.1*a0 + b <= -1
3.0*a1 + b <= -1
5.9*a2 + b <= -1
2.1*a3 +

# Question 2 (15 pts)

Implement the ADABoost algorithm by using the scikit implementation of the logistic regression. Evaluate the result on a real dataset between a single logistic regression and AdaBoost (use K-Fold cross validation).

This links can be helpful: http://rob.schapire.net/papers/explaining-adaboost.pdf and https://en.wikipedia.org/wiki/AdaBoost


In [3]:
cancer = load_breast_cancer()
df = pd.DataFrame(np.c_[cancer['data'], cancer['target']],
                  columns= np.append(cancer['feature_names'], ['target']))
X=cancer['data']
y=cancer['target']

def out(s):
    if s==1:
        return 1
    return -1
y=np.array([out(d) for d in y])


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
#scale the data
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [5]:
def adaboost_train(train_data,classes,iterations_num):
    #calculate number of samples in train data
    abselon=.000000000001 #small value to add to dominator at division
    best_classifiers=[]#list to save trained classifier
    calssifier_significance=[]#list to save significance of each classifier
    samples_num=train_data.shape[0]#number of samples in train data
    features_num=train_data.shape[1]#number of features in train data
    #define 1d numpy array of length == number of samples for saving sample weights
    weights=np.zeros(samples_num)#array to save the weight of each sample initialized to zero
    indices=np.zeros(samples_num)#array to save the index of each sample initialized to zero
    #initialize all weights equall with 1/samples_num
    weights=np.array([1/samples_num for d in weights])
    #add the index of each sample 
    for k in range(0,samples_num):
        indices[k] =k 
    #loop around iterations 
    for i in range(0,iterations_num):
        #train logistic regression
        clf = LogisticRegression(random_state=0).fit(train_data,classes)
        #pred the output for each sample 
        pred=clf.predict(train_data)
        error=0
        #calculate the error
        for k in range(0,samples_num):
            if (pred[k]!=y[k]):
                error=error+weights[k]
        #calculate significance
        significance=.5*np.log((1-error)/(error+abselon))
        #add classifier to list 
        best_classifiers.append(clf)
        #add significance to list 
        calssifier_significance.append(significance)
        #update the weights 
        for h in range(0,samples_num):
            if (pred[h]!=y[h]):
                weights[h]=weights[h]*np.exp(significance)
            else:
                weights[h]=weights[h]*np.exp(-1*significance)
        weights=weights/sum(weights)
        #based on the weights and indices we choose new dataset that sample with higher weights has most likelihood to be repeated in array
        indices=np.random.choice(indices,p=weights, size=(samples_num,),replace=True).astype(int)
        #update train data and classes with new indicies 
        train_data=train_data[indices]
        classes=classes[indices]
        #update the weights to be equal 
        weights=np.array([1/samples_num for d in weights])
        #make the indices for new array to be between 0-sample_num
        for k in range(0,samples_num):
            indices[k] =k
    #return two lists one for classifiers and one for indecies
    return calssifier_significance,best_classifiers



In [6]:
def adaboost_test(test_data,calssifier_significance,classifiers):
    #calculate number of data samples
    samples_num=test_data.shape[0]
    #initialize array for predictions
    predictions=np.zeros(samples_num)
    #loop around samples_number
    for i in range(0,samples_num):
        #read one test sample
        test_element=test_data[i,:].reshape(1, -1)
        calssifier_significance_num=0
        res=0
        #loop around classifier
        for obj in classifiers:
            #use the prediction of each classifier
            pred=obj.predict(test_element)
            sig=calssifier_significance[calssifier_significance_num]
            #multiply the prediciton of the classifier to its significance and add them to result
            res=res+(pred*sig)
            calssifier_significance_num=calssifier_significance_num+1
            #check if the final result >or < zero to take definal decision
        if(res>0):
            predictions[i]=1
        else:
            predictions[i]=-1
    return predictions


In [7]:
calssifier_significance,best_classifiers=adaboost_train(X_train,y_train,7)
predictions=adaboost_test(X_test,calssifier_significance,best_classifiers)
accuracy=accuracy_score(y_test, predictions)    
print(accuracy)

0.9868421052631579


In [8]:
clf = LogisticRegression().fit(X_train,y_train)
pred2=clf.predict(X_test)
accuracy=accuracy_score(y_test, pred2)    
print(accuracy)

0.9824561403508771


## Question 3 (10 pts)
Use DBscan (try with different parameters) and K-means (K=3) on IRIS Dataset and discuss/compare the results with the iris ground truth.
Please provide an explanation of why K-fold cross validation is not required for the comparison among these different algorithms.

In [9]:
data= pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',header=None)
X=data.iloc[:,[0,1,2,3]]
y=data.loc[:,4]
def out(s):
    if s=='Iris-versicolor':
        return 1
    elif s=='Iris-setosa':
        return 0
    return 2
y=np.array([out(d) for d in y])
X = X.to_numpy()

In [10]:
kmeans = KMeans(n_clusters=3, max_iter=1000).fit(X)
pred=kmeans.predict(X)
accuracy=homogeneity_score(y, pred)    
print(accuracy)

0.7514854021988338


In [11]:
eps=[.1,.2,.3,.4,.5,.6,.7,.8,.9,1,1.1,1.2,1.3,1.5,1.7,2]
min_samples=[2,3,4,5,6,7,8,9,10,15,20,25,30,35,40,45,50]
best_accuracy=0
best_eps=0
best_samples=0
for obj in eps:
    for samples in min_samples:
        pred = DBSCAN(eps=obj, min_samples=samples).fit_predict(X)
        accuracy=homogeneity_score(y, pred)
        if (accuracy>best_accuracy):
            best_accuracy=accuracy
            best_eps=obj
            best_samples=samples
print("Best parameters")        
print("epsilon=",best_eps)
print("min_samples=",best_samples)
print("accuracy=",best_accuracy)


Best parameters
epsilon= 0.4
min_samples= 2
accuracy= 0.8555893610823645


As the homogenity of DBscan is better than k-means this means that the points of each class can be considered to have density based structure.
Homogenity means each cluster contains only members of a single class. so if all the samples in the same class belonging to the same cluster the homogenity is 1 and if all samples in cluster each belongs to different classes
We assume that there are M data samples, J different class labels, K clusters and $\alpha_{ck}$ number of data-points belonging to the class c and cluster k Then the homogeneity h is given by the following
$h=1-(\frac{H(J,K)}{H(J)})$ 

where

$H(J,K)=-\sum \limits _{k=1} ^{K}\sum \limits _{j=1} ^{J}\frac{\alpha_{jk}}{M}log(\frac{\alpha_{jk}}{{\sum \limits _{j=1} ^{J} {\alpha_{jk}}}}) $

where $H(J,K)$ is the conditional entropy of the classes given the cluster assignments

$H(J)=-\sum \limits _{j=1} ^{J}\frac{\sum \limits _{k=1} ^{K}\alpha_{jk}}{C}log(\frac{\sum \limits _{k=1} ^{K}\alpha_{jk}}{J}) $

$H(J)$ is the entropy of the classes 


K-fold cross validation is used in classification to prevent overfitting and choosing hyperparameters but in clustering we don't feed to the model the labels, and feeding all the train data at once would help the algorithm to calculate its parameters such as the mean of each class in k-means more accurately than applying each fold and calculating the parameters for each fold and then doing the average of the results of all folds

# Extra Points ( 5pts)
An unbalanced dataset (e.g. 95% vs 5%) can be problematic even in the training phase. The learned function can be trivial, e.g. always predicting one class.
A possible solution can have a weight for each point in the way that making a mistake in the minority class will coun more w.r.t. the other. Please redefine the likelihood of the logistic regression to consider these weights for each point. Please compute the log-likelihood and its derivatives.
In addition, add to the negative log-likelihood the norm of W (sum of the square of each component) and compute the derivatives.

Likelihood function for the logistic regression function is given as:

$$\prod_{i=1}^{m} P(y = 1 | x^{(i)})^{y^{(i)}} P(y = 0 | x^{(i)})^{1 - y^{(i)}}$$

Without loss of generality, suppose that the class 1 has $k$ number of observations and class 0 has $m - k$ number of observations such that $k > m-2$. Then, each observation in class 2 should be counted as $\frac{k}{m-k}$ observation.

For instance,in the data set of $m = 10$ observations, suppose the class 1 has $k = 8$ observations and class 0 has $m - k = 2$ observations. Then each observation in class 0 should be counted as $\frac{8}{10-8} = 5$ observations to balance the data set.

Then the likelihood function of the logistic regression function is:

$$\prod_{i=1}^{m} P(y = 1 | x^{(i)})^{y^{(i)}} \left( \frac{k}{m-k} P(y = 0 | x^{(i)}) \right)^{1 - y^{(i)}}$$

and the log likelihood function of the logistic regression function would then be:

$$\sum_{i=1}^{m} y^{(i)} \log P(y = 1 | x^{(i)}) + \frac{k(1 - y^{(i)})}{m-k} \log P(y = 0 | x^{(i)})$$


This will ensure that the contribution to the log likelihood function is equally divided between two classes 0 and 1.